# VERIFICATION TESTING

# HER2 One Scanner - Hamamatsu2

- 5-Fold (80/20) split, No Holdout Set
- Truth = Categorical from Mean of 7 continuous scores 
- Epoch at automatic Stop when loss<.001 change 
- LeNet model, 10 layers, Dropout (0.7)

In [1]:
import numpy as np
import pandas as pd
import random
from keras.callbacks import EarlyStopping
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc, classification_report
import csv
import cv2
import scipy
import os
%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.
/usr/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#For single scanner
BASE_PATH = '/home/diam/Desktop/1Scanner_VerificationTest_HER2data/Hamamatsu2/'
#BASE PATH for working from home:
#BASE_PATH = '/home/OSEL/Desktop/HER2_data_categorical/'
#epochs = 10
batch_size = 32
num_classes = 3
#epochs = 35

## Get Data - Practice

In [3]:
#This is the version from Ravi's code:

#FDA
#X_FDA = []
#idx_FDA = []
#for index, image_filename in list(enumerate(BASE_PATH)):
#	img_file = cv2.imread(BASE_PATH + '/' + image_filename)
#	if img_file is not None:
		#img_file = smisc.imresize(arr = img_file, size = (600,760,3))
#		img_file = smisc.imresize(arr = img_file, size = (120,160,3))		
#		img_arr = np.asarray(img_file)
#		X_FDA.append(img_arr)
#		idx_FDA.append(index)

#X_FDA = np.asarray(X_FDA)
#idx_FDA = np.asarray(idx_FDA)
#random.seed(rs)
#random_id = random.sample(idx_FDA, len(idx_FDA)/2)
#random_FDA = []
#for i in random_id:
#	random_FDA.append(X_FDA[i])

#random_FDA = np.asarray(random_FDA)

## Get Data - Real

In [4]:
def get_data(folder):
    X = []
    y = []
    filenames = []

    for hclass in os.listdir(folder):
        if not hclass.startswith('.'):
            if hclass in ["1"]:
                label = 1
            else: #label must be 1 or 2
                if hclass in ["2"]:
                    label = 2
                else:
                    label = 3
            for image_filename in os.listdir(folder + hclass):
                filename = folder + hclass + '/' + image_filename
                img_file = cv2.imread(folder + hclass + '/' + image_filename)
                
                if img_file is not None:
                    img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
                    filenames.append(filename)

    X = np.asarray(X)
    y = np.asarray(y)
    z = np.asarray(filenames)
    return X,y,filenames

In [5]:
X, y, z = get_data(BASE_PATH)


#print(X)
#print(y)
#print(z)
print(len(X))
print(len(y))
print(len(z))

#INTEGER ENCODE
#https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
encoder = LabelEncoder()
y_cat = np_utils.to_categorical(encoder.fit_transform(y))
print(y_cat)





/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


241
241
241
[[ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.

### Old Code

In [6]:

#encoder = LabelEncoder()
#encoder.fit(y)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

#encoded_y_train = encoder.transform(y_train)
#encoded_y_test = encoder.transform(y_test)

#y_train = np_utils.to_categorical(encoded_y_train)
#y_test = np_utils.to_categorical(encoded_y_test)

In [7]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Fit Model with K-Fold X-Val

In [8]:
kf = KFold(n_splits = 5, shuffle=True)
print(kf.get_n_splits(y_cat))
print(kf)


#for train_index, test_index in kf.split(y):
#    X_train, X_test = X[train_index], X[test_index]
#    print(train_index, test_index)




5
KFold(n_splits=5, random_state=None, shuffle=True)


In [9]:

oos_y = []
oos_pred = []
fold = 0


for train, test in kf.split(y_cat):
    fold+=1
    print("fold #{}".format(fold))
        
    X_train = X[train]
    y_train = y_cat[train]
    X_test = X[test]
    y_test = y_cat[test]
    
    #encoder = LabelEncoder()
    #encoder.fit(y_test)
    #y_train = np_utils.to_categorical(encoder.transform(y_train))
    #y_test = np_utils.to_categorical(encoder.transform(y_test))
    
    model = Sequential()
    model.add(Lambda(lambda x: x * 1./255., input_shape=(120, 160, 3), output_shape=(120, 160, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(120, 160, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
    
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=25, verbose=1, mode='auto')
    
    model.fit(
            X,
            y_cat,
            validation_data=(X_test,y_test),
            callbacks=[monitor],
            shuffle=True,
            batch_size=batch_size,
            verbose=1,
            epochs=1000)
        
    pred = model.predict(X_test)
        
    oos_y.append(y_test)
    pred = np.argmax(pred,axis=1)
    oos_pred.append(pred)
    
        #measure the fold's accuracy
    y_compare = np.argmax(y_test,axis=1) #for accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)
    print("Fold Score (accuracy): {}".format(score))
    print(pred)

fold #1
Train on 241 samples, validate on 49 samples
Epoch 1/1000
241/241 [==============================] - 2s 9ms/step - loss: 1.2427 - acc: 0.4191 - val_loss: 0.9980 - val_acc: 0.8367
Epoch 2/1000
241/241 [==============================] - 0s 1ms/step - loss: 1.0082 - acc: 0.5145 - val_loss: 0.8880 - val_acc: 0.5510
Epoch 3/1000
241/241 [==============================] - 0s 1ms/step - loss: 1.0069 - acc: 0.5145 - val_loss: 0.9534 - val_acc: 0.5510
Epoch 4/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9974 - acc: 0.5643 - val_loss: 0.8436 - val_acc: 0.6735
Epoch 5/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9865 - acc: 0.5311 - val_loss: 0.9026 - val_acc: 0.5510
Epoch 6/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9872 - acc: 0.5643 - val_loss: 0.9068 - val_acc: 0.5510
Epoch 7/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9664 - acc: 0.5809 - val_loss: 0.9428 - val_acc: 0.6531
Epoch 8/1

241/241 [==============================] - 0s 1ms/step - loss: 0.2899 - acc: 0.8672 - val_loss: 0.0842 - val_acc: 0.9592
Epoch 62/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2885 - acc: 0.8589 - val_loss: 0.0946 - val_acc: 0.9592
Epoch 63/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2171 - acc: 0.9129 - val_loss: 0.0890 - val_acc: 0.9592
Epoch 64/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2520 - acc: 0.8755 - val_loss: 0.1225 - val_acc: 0.9592
Epoch 65/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3023 - acc: 0.8797 - val_loss: 0.0633 - val_acc: 0.9592
Epoch 66/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2573 - acc: 0.8838 - val_loss: 0.1438 - val_acc: 0.9592
Epoch 67/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2285 - acc: 0.8921 - val_loss: 0.1564 - val_acc: 0.9592
Epoch 68/1000
241/241 [==============================] - 0s 1ms/step 

241/241 [==============================] - 0s 1ms/step - loss: 0.1089 - acc: 0.9627 - val_loss: 0.0144 - val_acc: 1.0000
Epoch 122/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0926 - acc: 0.9668 - val_loss: 0.0198 - val_acc: 1.0000
Epoch 123/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1247 - acc: 0.9627 - val_loss: 0.0100 - val_acc: 1.0000
Epoch 124/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0782 - acc: 0.9710 - val_loss: 0.0069 - val_acc: 1.0000
Epoch 125/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1639 - acc: 0.9627 - val_loss: 0.0255 - val_acc: 1.0000
Epoch 126/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1070 - acc: 0.9710 - val_loss: 0.0577 - val_acc: 0.9592
Epoch 127/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1071 - acc: 0.9502 - val_loss: 0.0190 - val_acc: 1.0000
Epoch 128/1000
241/241 [==============================] - 0s 1m

241/241 [==============================] - 0s 2ms/step - loss: 0.0635 - acc: 0.9793 - val_loss: 0.0031 - val_acc: 1.0000
Epoch 182/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0549 - acc: 0.9834 - val_loss: 6.9005e-04 - val_acc: 1.0000
Epoch 183/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2540 - acc: 0.9212 - val_loss: 0.0103 - val_acc: 1.0000
Epoch 184/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0624 - acc: 0.9793 - val_loss: 0.0064 - val_acc: 1.0000
Epoch 185/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0827 - acc: 0.9668 - val_loss: 0.0013 - val_acc: 1.0000
Epoch 186/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0593 - acc: 0.9710 - val_loss: 0.0023 - val_acc: 1.0000
Epoch 187/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1725 - acc: 0.9419 - val_loss: 0.0056 - val_acc: 1.0000
Epoch 00187: early stopping
Fold Score (accuracy): 1.0
[1 1

241/241 [==============================] - 0s 1ms/step - loss: 0.1642 - acc: 0.9295 - val_loss: 0.1755 - val_acc: 0.8958
Epoch 54/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1624 - acc: 0.9253 - val_loss: 0.1288 - val_acc: 0.9375
Epoch 55/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1737 - acc: 0.8963 - val_loss: 0.1671 - val_acc: 0.9792
Epoch 56/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4501 - acc: 0.8506 - val_loss: 0.1070 - val_acc: 0.8958
Epoch 57/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1248 - acc: 0.9419 - val_loss: 0.1133 - val_acc: 0.9167
Epoch 58/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1275 - acc: 0.9336 - val_loss: 0.8684 - val_acc: 0.8958
Epoch 59/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3262 - acc: 0.9046 - val_loss: 0.0912 - val_acc: 1.0000
Epoch 60/1000
241/241 [==============================] - 0s 1ms/step 

...

Epoch 66/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1594 - acc: 0.9046 - val_loss: 0.1707 - val_acc: 0.9583
Epoch 67/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2214 - acc: 0.9046 - val_loss: 0.1429 - val_acc: 0.8958
Epoch 68/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5041 - acc: 0.8091 - val_loss: 0.1559 - val_acc: 0.9375
Epoch 69/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.2447 - acc: 0.8921 - val_loss: 0.0774 - val_acc: 0.9583
Epoch 70/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1774 - acc: 0.8963 - val_loss: 0.0959 - val_acc: 1.0000
Epoch 71/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1555 - acc: 0.9336 - val_loss: 0.0657 - val_acc: 0.9583
Epoch 72/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2297 - acc: 0.8963 - val_loss: 0.0783 - val_acc: 0.9583
Epoch 73/1000
241/241 [==============================] 

241/241 [==============================] - 0s 2ms/step - loss: 0.0669 - acc: 0.9668 - val_loss: 0.0043 - val_acc: 1.0000
Epoch 127/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1112 - acc: 0.9336 - val_loss: 0.0065 - val_acc: 1.0000
Epoch 128/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1022 - acc: 0.9544 - val_loss: 0.0062 - val_acc: 1.0000
Epoch 129/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0617 - acc: 0.9751 - val_loss: 0.0092 - val_acc: 1.0000
Epoch 130/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.6513 - acc: 0.8340 - val_loss: 0.0109 - val_acc: 1.0000
Epoch 131/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0866 - acc: 0.9668 - val_loss: 0.0068 - val_acc: 1.0000
Epoch 132/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0678 - acc: 0.9668 - val_loss: 0.0892 - val_acc: 0.9583
Epoch 133/1000
241/241 [==============================] - 0s 1m

Epoch 13/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9104 - acc: 0.7012 - val_loss: 0.8160 - val_acc: 0.6875
Epoch 14/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.7169 - acc: 0.7759 - val_loss: 0.6880 - val_acc: 0.7292
Epoch 15/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.7434 - acc: 0.7510 - val_loss: 0.6620 - val_acc: 0.7292
Epoch 16/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8608 - acc: 0.6971 - val_loss: 0.8513 - val_acc: 0.6250
Epoch 17/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.7582 - acc: 0.7593 - val_loss: 0.7294 - val_acc: 0.6875
Epoch 18/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.6351 - acc: 0.7593 - val_loss: 0.6466 - val_acc: 0.7500
Epoch 19/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.7514 - acc: 0.7593 - val_loss: 0.7102 - val_acc: 0.6875
Epoch 20/1000
241/241 [==============================] 

241/241 [==============================] - 0s 1ms/step - loss: 0.2252 - acc: 0.9004 - val_loss: 0.1552 - val_acc: 0.9167
Epoch 74/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1442 - acc: 0.9336 - val_loss: 0.1779 - val_acc: 0.8542
Epoch 75/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1481 - acc: 0.9378 - val_loss: 0.1722 - val_acc: 0.8333
Epoch 76/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1547 - acc: 0.9212 - val_loss: 0.1550 - val_acc: 0.8333
Epoch 77/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4524 - acc: 0.8672 - val_loss: 0.2387 - val_acc: 0.8333
Epoch 78/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2033 - acc: 0.9004 - val_loss: 0.1338 - val_acc: 0.9375
Epoch 79/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1383 - acc: 0.9461 - val_loss: 0.1357 - val_acc: 0.8958
Epoch 80/1000
241/241 [==============================] - 0s 1ms/step 

241/241 [==============================] - 0s 1ms/step - loss: 0.0659 - acc: 0.9793 - val_loss: 0.0088 - val_acc: 1.0000
Epoch 134/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0304 - acc: 0.9917 - val_loss: 0.0040 - val_acc: 1.0000
Epoch 135/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0606 - acc: 0.9751 - val_loss: 0.0125 - val_acc: 1.0000
Epoch 136/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0879 - acc: 0.9544 - val_loss: 0.0239 - val_acc: 1.0000
Epoch 137/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0503 - acc: 0.9751 - val_loss: 0.0025 - val_acc: 1.0000
Epoch 138/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3350 - acc: 0.8838 - val_loss: 0.0598 - val_acc: 0.9792
Epoch 139/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1081 - acc: 0.9461 - val_loss: 0.0146 - val_acc: 1.0000
Epoch 140/1000
241/241 [==============================] - 0s 2m